In [1]:
import openai
import pandas as pd
import numpy as np
import json
import os
import re
import time
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv, find_dotenv

In [2]:
os.chdir('/Users/chrissoria/Documents/Research/UCNets_Classification/Hand_Coding_Surveys/e2b')
current_directory = os.getcwd()
print(current_directory)

/Users/chrissoria/Documents/Research/UCNets_Classification/Hand_Coding_Surveys/e2b


In [3]:
Chris = pd.read_excel("e2ab_Master.xlsx", engine='openpyxl', sheet_name="CF_CORRECTED_9.14.23").drop(["Response"], axis=1)
Claude = pd.read_excel("e2ab_Master.xlsx", engine='openpyxl', sheet_name="e2b_CF").drop(["Comments", "Response","prim_key","global comments"], axis=1)

In [4]:
Chris = Chris.fillna(0)
Claude = Claude.fillna(0)

In [5]:
# Define a custom function to remove decimals and convert to integers
def remove_decimals(value):
    if isinstance(value, (int, float)):
        return int(value)
    else:
        return value

# Apply the custom function to each element in the DataFrame
Chris = Chris.applymap(remove_decimals)
Claude = Claude.applymap(remove_decimals)

In [6]:
Claude.head()

,Simple Assertion,Implicit Friendship Norms,Explicit Friendship Norms,Close-knit Loyalty,Personal Character,Past Support,Unreliable,Reciprocal Support,Estrangement and Hostility,Independence or Unwillingness,Friend's Resources,Geographic Proximity,Friends as Family,No Friends,Verbal Assurance,Duration of Relationship,Other Reasons for Help,Other Reasons for NO Help,Not Clear / Uncodeable
0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


After going over the discrepencies with Claude, we decided that
- Simple Assertion:   Gets code Yes IFF it is coded Yes by EITHER coder AND has no other codes.

In [7]:
Claude['column_sums'] = Claude.iloc[:, 1:-3].sum(axis=1)
Chris['column_sums'] = Chris.iloc[:, 1:].sum(axis=1)

Claude['Simple_Assertion'] = Claude.apply(lambda row: 0 if row['column_sums'] > 0 and row['Simple Assertion'] == 1 else row['Simple Assertion'], axis=1)
Chris['Simple_Assertion'] = Chris.apply(lambda row: 0 if row['column_sums'] > 0 and row['Simple Assertion'] == 1 else row['Simple Assertion'], axis=1)

Claude['Simple_Assertion'] = Claude.apply(lambda row: 1 if row['Simple_Assertion'] == 1 or Chris.loc[row.name, 'Simple_Assertion'] == 1 else row['Simple_Assertion'], axis=1)
Claude['Past_Support'] = Claude.apply(lambda row: 0 if row['column_sums'] > 0 and row['Simple_Assertion'] == 1 else row['Simple_Assertion'], axis=1)

Claude = Claude.drop(columns=['column_sums','Simple Assertion'])
Chris = Chris.drop(columns=['column_sums','Simple Assertion'])

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_41644/3353353876.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  Chris['column_sums'] = Chris.iloc[:, 1:].sum(axis=1)


Explicit : EITHER a CS or a CF Yes –> Yes \
Close-Knit: EITHER \
Personal Character: EITHER \
Past Support: EITHER \
Unreliable: EITHER. \
     > Note: This is a very unreliable coding; many differences between CF and CS, and has to be “asterisked” in future \
Reciprocal: EITHER \
Estranged: EITHER \
     > In re-labeling, have to make clear that this is “indifference, estrangement, or hostility.” \
Independent: EITHER \
Resources: EITHER \
     > Understood to largely involve references to material resources, but also more broadly coded (as CF did) to include resources like time, demands... It’s about constraints on friends’ ability to help. \
Geography (Distance): EITHER \
Friends as Family: EITHER \
No Friends: EITHER \
     > Understood as few or no friends; emphasis by respondent an scarcity/absence of friends. \
Verbal: EITHER \
Other, Yes: EITHER \
Other, No: EITHER

$$
v_{N,i} = \begin{cases} 
1 & \text{if } v_{C,i} = 1 \text{ or } v_{K,i} = 1 \\
v_{C,i} & \text{otherwise}
\end{cases}
$$

In [8]:
Claude['Explicit_Friendship_Norms'] = Claude.apply(lambda row: 1 if row['Explicit Friendship Norms'] == 1 or Chris.loc[row.name, 'Explicit Friendship Norms'] == 1 else row['Explicit Friendship Norms'], axis=1)
Claude['Close_knit'] = Claude.apply(lambda row: 1 if row['Close-knit Loyalty'] == 1 or Chris.loc[row.name, 'Close-knit Loyalty'] == 1 else row['Close-knit Loyalty'], axis=1)
Claude['Personal_Character'] = Claude.apply(lambda row: 1 if row['Personal Character'] == 1 or Chris.loc[row.name, 'Personal Character'] == 1 else row['Personal Character'], axis=1)
Claude['Past_Support'] = Claude.apply(lambda row: 1 if row['Past Support'] == 1 or Chris.loc[row.name, 'Past Support'] == 1 else row['Past Support'], axis=1)
Claude['Not_Reliable'] = Claude.apply(lambda row: 1 if row['Unreliable '] == 1 or Chris.loc[row.name, 'Unreliable '] == 1 else row['Unreliable '], axis=1)
Claude['Reciprocal_Support'] = Claude.apply(lambda row: 1 if row['Reciprocal Support '] == 1 or Chris.loc[row.name, 'Reciprocal Support '] == 1 else row['Reciprocal Support '], axis=1)
Claude['Estrangement'] = Claude.apply(lambda row: 1 if row['Estrangement and Hostility'] == 1 or Chris.loc[row.name, 'Estrangement and Hostility'] == 1 else row['Estrangement and Hostility'], axis=1)
Claude['Independence'] = Claude.apply(lambda row: 1 if row['Independence or Unwillingness'] == 1 or Chris.loc[row.name, 'Independence or Unwillingness'] == 1 else row['Independence or Unwillingness'], axis=1)
Claude['Resources'] = Claude.apply(lambda row: 1 if row["Friend's Resources"] == 1 or Chris.loc[row.name, "Friend's Resources"] == 1 else row["Friend's Resources"], axis=1)
Claude['Geographic_Proximity'] = Claude.apply(lambda row: 1 if row["Geographic Proximity"] == 1 or Chris.loc[row.name, "Geographic Proximity"] == 1 else row["Geographic Proximity"], axis=1)
Claude['Friends_as_Family'] = Claude.apply(lambda row: 1 if row["Friends as Family"] == 1 or Chris.loc[row.name, "Friends as Family"] == 1 else row["Friends as Family"], axis=1)
Claude['No_Friends'] = Claude.apply(lambda row: 1 if row["No Friends"] == 1 or Chris.loc[row.name, "No Friends"] == 1 else row["No Friends"], axis=1)
Claude['Verbal'] = Claude.apply(lambda row: 1 if row["Verbal Assurance"] == 1 or Chris.loc[row.name, "Verbal Assurance"] == 1 else row["Verbal Assurance"], axis=1)
Claude['Other_Help'] = Claude.apply(lambda row: 1 if row["Other Reasons for Help"] == 1 or Chris.loc[row.name, "Other Reasons for Help"] == 1 else row["Other Reasons for Help"], axis=1)
Claude['Other_no_Help'] = Claude.apply(lambda row: 1 if row["Other Reasons for NO Help"] == 1 or Chris.loc[row.name, "Other Reasons for NO Help"] == 1 else row["Other Reasons for NO Help"], axis=1)

Claude = Claude.drop(columns=['Explicit Friendship Norms','Close-knit Loyalty','Personal Character','Past Support',
                             'Unreliable ','Reciprocal Support ','Estrangement and Hostility','Independence or Unwillingness',
                             "Friend's Resources",'Geographic Proximity','Friends as Family','No Friends','Other Reasons for Help',
                             'Other Reasons for NO Help','Verbal Assurance'])

Duration: Use Chris’s codes.

In [9]:
Claude['Duration'] = Chris['Duration of Relationship']
Claude = Claude.drop(columns=['Duration of Relationship'])

Implicit: Use Claude's codes. In other words we'll just leave that column alone.

In [10]:
Claude['Implicit_Friendship_Norms'] = Claude['Implicit Friendship Norms']
Claude = Claude.drop(columns=['Implicit Friendship Norms'])

In [11]:
ordered_columns = ['Simple_Assertion', 'Implicit_Friendship_Norms', 'Explicit_Friendship_Norms', 'Close_knit',
                  'Personal_Character','Past_Support','Not_Reliable','Reciprocal_Support','Estrangement','Independence',
                  'Resources','Geographic_Proximity','Friends_as_Family','No_Friends','Verbal','Duration',
                  'Other_Help','Other_no_Help','Not Clear / Uncodeable']

Claude = Claude[ordered_columns]

Next, Claude and I agreed that we would create new columns
- life experiences 
- preference for family 
- reciprocal: would 
- single person

In [12]:
Chris_comments = pd.read_excel("e2ab_Master.xlsx", engine='openpyxl', sheet_name="e2b_CS")[["Comments"]]['Comments'].str.lower()
Claude_comments = pd.read_excel("e2ab_Master.xlsx", engine="openpyxl", sheet_name="e2b_CF")[["Comments"]]['Comments'].str.lower()

Claude['Chris_comments'] = Chris_comments.astype(str)
Claude['Claude_comments'] = Claude_comments.astype(str)

Claude['Life_Experience'] = Claude['Chris_comments'].str.contains('experience').astype(int)
Claude['Family_First'] = Claude['Chris_comments'].str.contains('preference').astype(int)
Claude['Reciprocal_Would'] = Claude['Claude_comments'].str.contains('reciprocal: would').astype(int)
Claude['Single_Person'] = Claude['Claude_comments'].str.contains('single person').astype(int)

/Users/chrissoria/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [13]:
Claude['Other_Help'] = Claude.apply(lambda row: 0 if row['Life_Experience'] == 1 or row['Family_First'] == 1 or row['Reciprocal_Would'] == 1 or row['Single_Person'] == 1 else row['Other_Help'], axis=1)

In [14]:
ordered_columns = ['Simple_Assertion', 'Implicit_Friendship_Norms', 'Explicit_Friendship_Norms', 'Close_knit',
                  'Personal_Character','Past_Support','Not_Reliable','Reciprocal_Support','Estrangement','Independence',
                  'Resources','Geographic_Proximity','Friends_as_Family','No_Friends','Verbal','Duration','Life_Experience',
                  'Family_First','Reciprocal_Would','Single_Person','Other_Help','Other_no_Help','Not Clear / Uncodeable']

Claude = Claude[ordered_columns]

In [15]:
Claude['Uncodeable'] = np.where(Claude.iloc[:, :18].sum(axis=1) > 0, 0, Claude['Not Clear / Uncodeable'])

In [17]:
# Read the response column from the Excel file
response = pd.read_excel("e2ab_Master.xlsx", engine='openpyxl', sheet_name="e2b_CS", usecols="A")

# Rename the column to "Response"
response = response.rename(columns={"A": "prim_key"})

Claude = pd.concat([response['prim_key'].reset_index(drop=True), Claude.reset_index(drop=True)], axis=1)

Claude.to_csv("/Users/chrissoria/Documents/Research/Family_Versus_Friend_Social_Support/codes/e2ab_Master_Coded.csv", index=False)